# Test Generator for Classes

This notebook provides a structured approach to generate comprehensive test suites for classes. It covers basic testing, edge cases, exception handling, and test organization.

## Import Required Libraries

Import unittest, pytest, or other testing frameworks needed for testing the class.

In [ ]:
# Import testing frameworks
import unittest
import pytest
import inspect
import coverage
import sys
import os
from typing import Any, Dict, List, Optional, Tuple, Union, Callable

## Define the Class to Test

Create or import the class that needs to be tested. Include method signatures and basic functionality.

In [ ]:
# Sample class to test - replace with your own class or import your class
class Calculator:
    """A simple calculator class to demonstrate testing."""
    
    def __init__(self):
        self.result: float = 0.0
        self.history: List[str] = []
    
    def add(self, x: float, y: float) -> float:
        """Add two numbers and store the result."""
        self.result = x + y
        self.history.append(f"{x} + {y} = {self.result}")
        return self.result
    
    def subtract(self, x: float, y: float) -> float:
        """Subtract y from x and store the result."""
        self.result = x - y
        self.history.append(f"{x} - {y} = {self.result}")
        return self.result
    
    def multiply(self, x: float, y: float) -> float:
        """Multiply two numbers and store the result."""
        self.result = x * y
        self.history.append(f"{x} * {y} = {self.result}")
        return self.result
    
    def divide(self, x: float, y: float) -> float:
        """Divide x by y and store the result."""
        if y == 0:
            raise ValueError("Cannot divide by zero")
        self.result = x / y
        self.history.append(f"{x} / {y} = {self.result}")
        return self.result
    
    def clear(self) -> None:
        """Clear the current result and history."""
        self.result = 0.0
        self.history = []
    
    def get_history(self) -> List[str]:
        """Return calculation history."""
        return self.history

## Write Basic Unit Tests

Create test cases for the basic functionality of the class, testing each method with normal inputs.

In [ ]:
class TestCalculatorBasic(unittest.TestCase):
    """Basic test cases for the Calculator class."""
    
    def setUp(self):
        """Set up a new Calculator instance before each test."""
        self.calc = Calculator()
    
    def test_add(self):
        """Test the add method with typical inputs."""
        self.assertEqual(self.calc.add(2, 3), 5)
        self.assertEqual(self.calc.add(-1, 1), 0)
        self.assertEqual(self.calc.add(0, 0), 0)
    
    def test_subtract(self):
        """Test the subtract method with typical inputs."""
        self.assertEqual(self.calc.subtract(5, 2), 3)
        self.assertEqual(self.calc.subtract(1, 1), 0)
        self.assertEqual(self.calc.subtract(0, 5), -5)
    
    def test_multiply(self):
        """Test the multiply method with typical inputs."""
        self.assertEqual(self.calc.multiply(3, 4), 12)
        self.assertEqual(self.calc.multiply(-2, 3), -6)
        self.assertEqual(self.calc.multiply(0, 5), 0)
    
    def test_divide(self):
        """Test the divide method with typical inputs."""
        self.assertEqual(self.calc.divide(10, 2), 5)
        self.assertEqual(self.calc.divide(7, 2), 3.5)
        self.assertEqual(self.calc.divide(-6, 3), -2)
    
    def test_history(self):
        """Test that history is correctly recorded."""
        self.calc.add(2, 3)
        self.calc.subtract(10, 5)
        history = self.calc.get_history()
        self.assertEqual(len(history), 2)
        self.assertEqual(history[0], "2 + 3 = 5")
        self.assertEqual(history[1], "10 - 5 = 5")
    
    def test_clear(self):
        """Test the clear method."""
        self.calc.add(2, 3)
        self.calc.clear()
        self.assertEqual(self.calc.result, 0.0)
        self.assertEqual(len(self.calc.get_history()), 0)

## Test Edge Cases

Write tests for edge cases such as empty inputs, boundary values, and extreme inputs.

In [ ]:
class TestCalculatorEdgeCases(unittest.TestCase):
    """Edge case tests for the Calculator class."""
    
    def setUp(self):
        """Set up a new Calculator instance before each test."""
        self.calc = Calculator()
    
    def test_large_numbers(self):
        """Test with very large numbers."""
        large_num: float = 1e15
        self.assertEqual(self.calc.add(large_num, large_num), 2e15)
        self.assertEqual(self.calc.multiply(large_num, 2), 2e15)
    
    def test_small_numbers(self):
        """Test with very small decimal numbers."""
        small_num: float = 1e-15
        self.assertAlmostEqual(self.calc.add(small_num, small_num), 2e-15)
        self.assertAlmostEqual(self.calc.divide(small_num, 2), 5e-16)
    
    def test_float_precision(self):
        """Test floating point precision issues."""
        # Python's floating point arithmetic can sometimes be surprising
        self.assertAlmostEqual(self.calc.add(0.1, 0.2), 0.3, places=10)
    
    def test_negative_zeros(self):
        """Test with negative zeros in floating point."""
        negative_zero: float = -0.0
        self.assertEqual(self.calc.add(negative_zero, 0), 0)
        self.assertEqual(self.calc.multiply(negative_zero, 5), 0)
    
    def test_max_min_values(self):
        """Test with Python's maximum and minimum representable floats."""
        import sys
        max_float: float = sys.float_info.max
        min_float: float = sys.float_info.min
        
        # These operations should work without overflow/underflow
        self.calc.add(max_float, min_float)
        self.calc.divide(min_float, 2)

## Test Exception Handling

Create tests that verify the class properly handles exceptions and error conditions.

In [ ]:
class TestCalculatorExceptions(unittest.TestCase):
    """Exception handling tests for the Calculator class."""
    
    def setUp(self):
        """Set up a new Calculator instance before each test."""
        self.calc = Calculator()
    
    def test_divide_by_zero(self):
        """Test that division by zero raises ValueError."""
        with self.assertRaises(ValueError) as context:
            self.calc.divide(5, 0)
        
        self.assertEqual(str(context.exception), "Cannot divide by zero")
    
    def test_type_errors(self):
        """Test behavior when invalid types are provided."""
        with self.assertRaises(TypeError):
            self.calc.add("2", 3)  # type: ignore
        
        with self.assertRaises(TypeError):
            self.calc.multiply([], 3)  # type: ignore
    
    def test_operation_after_error(self):
        """Test that calculator still works after an error."""
        try:
            self.calc.divide(5, 0)
        except ValueError:
            pass
        
        # Calculator should still work after error
        self.assertEqual(self.calc.add(2, 3), 5)

## Create Test Suite

Organize all tests into a comprehensive test suite that can be run together.

In [ ]:
def create_test_suite() -> unittest.TestSuite:
    """Create a comprehensive test suite for the Calculator class."""
    suite = unittest.TestSuite()
    
    # Add test cases from each test class
    suite.addTest(unittest.makeSuite(TestCalculatorBasic))
    suite.addTest(unittest.makeSuite(TestCalculatorEdgeCases))
    suite.addTest(unittest.makeSuite(TestCalculatorExceptions))
    
    return suite

# Alternative approach using pytest
def generate_pytest_command() -> str:
    """Generate the command to run all tests with pytest."""
    # Save the test classes to a file
    test_file: str = "test_calculator.py"
    
    with open(test_file, "w") as f:
        f.write(inspect.getsource(Calculator) + "\n\n")
        f.write(inspect.getsource(TestCalculatorBasic) + "\n\n")
        f.write(inspect.getsource(TestCalculatorEdgeCases) + "\n\n")
        f.write(inspect.getsource(TestCalculatorExceptions) + "\n\n")
    
    # Return the command to run pytest
    return f"pytest {test_file} -v"

## Run Tests and Analyze Results

Execute the test suite and analyze the results, demonstrating how to interpret test coverage and failures.

In [ ]:
# Run the test suite
def run_test_suite() -> None:
    """Run the test suite and print results."""
    suite = create_test_suite()
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    print("\nTest Summary:")
    print(f"Ran {result.testsRun} tests")
    print(f"Failures: {len(result.failures)}")
    print(f"Errors: {len(result.errors)}")
    
    if result.failures:
        print("\nFailure Details:")
        for test, traceback in result.failures:
            print(f"\n{test}")
            print(traceback)
    
    if result.errors:
        print("\nError Details:")
        for test, traceback in result.errors:
            print(f"\n{test}")
            print(traceback)

# Run tests with code coverage
def run_tests_with_coverage() -> None:
    """Run tests with code coverage analysis."""
    cov = coverage.Coverage()
    cov.start()
    
    run_test_suite()
    
    cov.stop()
    cov.save()
    
    print("\nCoverage Report:")
    cov.report(show_missing=True)
    
    print("\nTo generate an HTML report, use:")
    print("cov.html_report(directory='htmlcov')")

# Execute the tests
run_test_suite()

In [ ]:
# Generate a test class template for any class
def generate_test_class_template(cls: Any) -> str:
    """
    Automatically generate a test class template for any given class.
    
    Args:
        cls: The class to generate tests for
    
    Returns:
        A string containing the test class definition
    """
    class_name: str = cls.__name__
    test_class_name: str = f"Test{class_name}"
    
    methods: List[str] = [name for name, method in inspect.getmembers(cls, inspect.isfunction) 
                         if not name.startswith('_')]
    
    test_class: str = f"class {test_class_name}(unittest.TestCase):\n"
    test_class += f"    \"\"\"Tests for the {class_name} class.\"\"\"\n\n"
    
    test_class += "    def setUp(self):\n"
    test_class += f"        \"\"\"Set up a new {class_name} instance before each test.\"\"\"\n"
    test_class += f"        self.instance = {class_name}()\n\n"
    
    for method in methods:
        test_class += f"    def test_{method}(self):\n"
        test_class += f"        \"\"\"Test the {method} method.\"\"\"\n"
        test_class += "        # TODO: Implement test for this method\n"
        test_class += "        pass\n\n"
    
    test_class += "    def test_edge_cases(self):\n"
    test_class += "        \"\"\"Test edge cases for this class.\"\"\"\n"
    test_class += "        # TODO: Implement edge case tests\n"
    test_class += "        pass\n\n"
    
    test_class += "    def test_exceptions(self):\n"
    test_class += "        \"\"\"Test exception handling for this class.\"\"\"\n"
    test_class += "        # TODO: Implement exception tests\n"
    test_class += "        pass\n"
    
    return test_class

# Example usage of the template generator
template = generate_test_class_template(Calculator)
print(template)

## Summary

This notebook demonstrated how to create a comprehensive test suite for a class, covering:

1. Basic functionality testing with typical inputs
2. Edge case testing with unusual inputs
3. Exception handling and error conditions
4. Organizing tests into a cohesive suite
5. Running tests and analyzing results
6. Automatically generating test templates

You can adapt this approach for your own classes by:
- Replacing the Calculator class with your own class
- Modifying the test cases to match your class's functionality
- Running the generated test suite to verify your implementation